# Threat Hunting
### Hunting `PowerShell Execution` 

```
@Author: SK
```

### Hypothesis
Adversaries leverage `PowerShell to execute code` within the environmen

### Tradecraft
Using PowerShell to perform a number of actions, including, discovery of information & code execution. Therefore hunting/analysis of powershell artifacts left when the powershell  is used in the environment.

### Initialization

In [1]:
## importing requsites
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [7]:
# Setting the spark log level to ERROR
spark.sparkContext.setLogLevel("ERROR")

# Creatig a spark session
spark = SparkSession.builder.master('local[*]').appName('Testing').getOrCreate()

In [8]:
# Set the output fields to 1000
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

# export the empire_launher file as a datframe to create a table
dataset = spark.read.json('empire_launcher_vbs_2020-09-04160940.json')

In [9]:
# Creating a temp table from empire_launcher file
dataset.createOrReplaceTempView('empire')

In [15]:
# Showing the scema of the table
print(dataset.columns)

# Showing the data of the table
dataset.show(30)

['@timestamp', '@version', 'AccessList', 'AccessMask', 'AccessReason', 'AccountName', 'AccountType', 'ActivityID', 'Application', 'Archived', 'AuthenticationPackageName', 'CallTrace', 'Category', 'Channel', 'CommandLine', 'Company', 'ContextInfo', 'CreationUtcTime', 'CurrentDirectory', 'Description', 'DestAddress', 'DestPort', 'DestinationHostname', 'DestinationIp', 'DestinationIsIpv6', 'DestinationPort', 'DestinationPortName', 'Details', 'Device', 'Direction', 'DisabledPrivilegeList', 'Domain', 'ERROR_EVT_UNRESOLVED', 'ElevatedToken', 'EnabledPrivilegeList', 'EventCountTotal', 'EventID', 'EventIdx', 'EventReceivedTime', 'EventTime', 'EventType', 'EventTypeOrignal', 'ExecutionProcessID', 'FileVersion', 'FilterRTID', 'GrantedAccess', 'GroupMembership', 'HandleId', 'Hashes', 'Hostname', 'Image', 'ImageLoaded', 'ImpersonationLevel', 'Initiated', 'IntegrityLevel', 'IpAddress', 'IpPort', 'IsExecutable', 'KeyLength', 'Keywords', 'LayerName', 'LayerRTID', 'LmPackageName', 'LogonGuid', 'LogonI